In [1]:
import csv
from itertools import combinations

In [2]:

def read_data(file_loc):
    trans = dict()
    with open(file_loc) as f:
        filedata = csv.reader(f, delimiter=',')
        count = 0
        for line in filedata:
            count += 1
            trans[count] = list(set(line))
    return trans
    

def frequence(items_lst, trans, check=False):
    items_counts = dict()
    for i in items_lst:
        temp_i = {i}
        if check:
            temp_i = set(i)
        for j in trans.items():
            if temp_i.issubset(set(j[1])):
                if i in items_counts:
                    items_counts[i] += 1
                else:
                    items_counts[i] = 1
    return items_counts
    

def support(items_counts, trans):
    support = dict()
    total_trans = len(trans)
    for i in items_counts:
        support[i] = items_counts[i]/total_trans
    return support
    
def association_rules(items_grater_then_min_support):
    rules = []
    dict_rules = {}
    for i in items_grater_then_min_support:
        dict_rules = {}
        if type(i) != type(str()):
            i = list(i)
            temp_i = i[:]
            for j in range(len(i)):
                k = temp_i[j]
                del temp_i[j]
                dict_rules[k] = temp_i
                temp_i = i[:]
        rules.append(dict_rules)
    temp = []
    for i in rules:
        for j in i.items():
            if type(j[1]) != type(str()):
                temp.append({tuple(j[1])[0]: j[0]})
            else:
                temp.append({j[1]: j[0]})
    rules.extend(temp)
    return rules

def confidence(associations, d, min_confidence):
    ans = {}
    for i in associations:
        for j in i.items():
            if type(j[0]) == type(str()):
                left = {j[0]}
            else:
                left = set(j[0])
            if type(j[1]) == type(str()):
                right = {j[1]}
            else:
                right = set(j[1])
            for k in d:
                if type(k) != type(str()):
                    if left.union(right) - set(k) == set():
                        up = d[k]
                    if len(right) == len(set(k)) and right - set(k) == set():
                            down = d[k]
                else:
                    if len(right) >= len({k}):
                        if right - {k} == set():
                            down = d[k]
                    elif len(right) <= len({k}):
                        if {k} - right == set():
                            down = d[k]
            if up/down >= min_confidence:
                ans[tuple(left)[0]] = right, up/down, up, down
    print(ans)    

In [6]:
min_support = 0.01
min_confidence = 0.7
trans = read_data("/home/magody/programming/python/data_science/data/transactions/grocery_store.csv")
number_of_trans = [len(i) for i in trans.values()]
items_lst = set()

itemcount_track = list()    

for i in trans.values():
    for j in i:
        items_lst.add(j)

store_item_lst = list(items_lst)[:]
items_grater_then_min_support = list()
items_counts = frequence(items_lst, trans)
print(len(trans))
print(store_item_lst)
print(items_counts)
itemcount_track.append(items_counts)
items_grater_then_min_support.append({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support})
print(items_grater_then_min_support)


20
['MILK', 'TEA', 'MAGGI', 'BOURNVITA', 'COCK', 'JAM', 'COFFEE', 'SUGER', 'BREAD', 'CORNFLAKES', 'BISCUIT']
{'MILK': 5, 'TEA': 7, 'MAGGI': 5, 'BOURNVITA': 4, 'COCK': 3, 'JAM': 2, 'COFFEE': 8, 'SUGER': 6, 'BREAD': 13, 'CORNFLAKES': 6, 'BISCUIT': 7}
[{'MILK': 0.25, 'TEA': 0.35, 'MAGGI': 0.25, 'BOURNVITA': 0.2, 'COCK': 0.15, 'JAM': 0.1, 'COFFEE': 0.4, 'SUGER': 0.3, 'BREAD': 0.65, 'CORNFLAKES': 0.3, 'BISCUIT': 0.35}]


In [10]:
for i in range(2, max(number_of_trans)+1):
    item_list = combinations(items_lst, i)
    items_counts = frequence(item_list, trans, check=True)
    itemcount_track.append(items_counts)
    if list({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support}.keys()) != []:
        items_grater_then_min_support.append({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support})
    
d = {}
{d.update(i) for i in itemcount_track}
associations = association_rules(items_grater_then_min_support[len(items_grater_then_min_support)-1])
associations_grater_then_confidene = confidence(associations, d, min_confidence)
print(associations)

{'MILK': ({'BREAD', 'CORNFLAKES', 'BISCUIT'}, 1.0, 1, 1), 'TEA': ({'BREAD', 'MAGGI', 'BISCUIT'}, 1.0, 1, 1), 'COFFEE': ({'CORNFLAKES', 'COCK', 'BISCUIT'}, 1.0, 2, 2), 'CORNFLAKES': ({'BISCUIT', 'COCK', 'COFFEE'}, 1.0, 2, 2), 'MAGGI': ({'TEA', 'BREAD', 'BISCUIT'}, 1.0, 1, 1), 'JAM': ({'BREAD', 'MAGGI', 'MILK'}, 1.0, 1, 1), 'BREAD': ({'JAM', 'TEA', 'MAGGI'}, 1.0, 1, 1), 'BISCUIT': ({'CORNFLAKES', 'COCK', 'COFFEE'}, 1.0, 2, 2), 'COCK': ({'BISCUIT', 'CORNFLAKES', 'COFFEE'}, 1.0, 2, 2)}
[{'MILK': ['TEA', 'COFFEE', 'CORNFLAKES'], 'TEA': ['MILK', 'COFFEE', 'CORNFLAKES'], 'COFFEE': ['MILK', 'TEA', 'CORNFLAKES'], 'CORNFLAKES': ['MILK', 'TEA', 'COFFEE']}, {'MILK': ['MAGGI', 'JAM', 'BREAD'], 'MAGGI': ['MILK', 'JAM', 'BREAD'], 'JAM': ['MILK', 'MAGGI', 'BREAD'], 'BREAD': ['MILK', 'MAGGI', 'JAM']}, {'MILK': ['BREAD', 'CORNFLAKES', 'BISCUIT'], 'BREAD': ['MILK', 'CORNFLAKES', 'BISCUIT'], 'CORNFLAKES': ['MILK', 'BREAD', 'BISCUIT'], 'BISCUIT': ['MILK', 'BREAD', 'CORNFLAKES']}, {'TEA': ['MAGGI', 'JAM', '